In [101]:
#Importing the necessary libraries

import os 
import sys
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import ticker
import numpy as np

import statsmodels.api as sm


In [102]:
#Global Settings 
pd.set_option('display.float_format', lambda x: '%.4f' % x)

In [103]:
#Importing Configs
# Define the path where config.py is located
#Mac
os.chdir('/Users/kylenabors/Documents/GitHub/Finance-ML-Modeling')
config_file_path = os.getcwd()
print(config_file_path)

# Add this path to the sys.path
sys.path.append(config_file_path)

import config

#Configs
finbert_models = config.finbert_models
Body = config.Body
database = config.Local_Database

/Users/kylenabors/Documents/GitHub/Finance-ML-Modeling


In [104]:
sentiment = pd.date_range(start='1/1/1990', end='1/1/2024', freq='D')
sentiment = pd.DataFrame(sentiment, columns=['date'])

state = pd.read_csv(f"{finbert_models}/{Body}/Statements/{Body}_Statements_finbert_model_short.csv")
state = state[['date','sentiment']]
state = state.rename(columns={'sentiment':'statement_sentiment'})
state['date'] = pd.to_datetime(state['date'])
sentiment = pd.merge(sentiment, state, how='left', left_on='date', right_on='date')

In [105]:
market =pd.read_csv(f'{database}/Market Data/All Market Data.csv')
market = market.rename(columns={'PCE':'pce', 'GDP':'gdp', 'GDPPOT':'gdp_pot'})
market['date'] = pd.to_datetime(market['date'])
print(market.columns)


Index(['date', 'sp500_price', 'sp500_volume', 'emini_price',
       'eurodollar_price', 'fedfutures_price', 'vix_sentiment', 'unemployment',
       'inflation', 'fedfunds', 'pce', 'gdp', 'gdp_pot'],
      dtype='object')


In [106]:
taylor = pd.read_csv(f'/Users/kylenabors/Documents/Database/Market Data/Taylor/Taylor.csv')
taylor['date'] = pd.to_datetime(taylor['date'])
taylor = taylor.rename({'Taylor':'taylor_fred'})

In [107]:

sentiment = pd.merge(sentiment, market, how='left', left_on='date', right_on='date')
sentiment = pd.merge(sentiment, taylor, how='left', left_on='date', right_on='date')

In [108]:
sentiment = sentiment[(sentiment['date'] >= '2000-01-01') & (sentiment['date'] <= '2023-12-30')]
sentiment = sentiment.set_index('date')
sentiment = sentiment.resample('Q').mean().reset_index()


In [109]:

filter_items = ['pce', 'gdp', 'gdp_pot', 'inflation']

for item in filter_items:
    filter_df = sentiment.copy(deep=True)
    filter_df = filter_df[['date', f'{item}']]

    cycle, trend = sm.tsa.filters.hpfilter(filter_df[f'{item}'], 1600)

    item_decomp = filter_df[[f'{item}']]
    item_decomp[f"{item}_cycle"] = cycle
    item_decomp[f"{item}_trend"] = trend

    filter_df = pd.merge(filter_df, item_decomp, on=f'{item}', how='left')  
    filter_df = filter_df[['date', f'{item}_cycle', f'{item}_trend']]
    sentiment = pd.merge(sentiment, filter_df, on='date', how='left')

In [110]:

sentiment['gap'] = (sentiment['gdp_trend'] - sentiment['gdp_pot_trend'])/sentiment['gdp_pot_trend']
sentiment['gap'] = sentiment['gap']*100

In [111]:
sentiment['taylor'] = sentiment['inflation_trend'] + sentiment['pce_trend'] + 0.5*(sentiment['pce_trend'] - 2) + 0.5*(sentiment['gap'])


In [112]:

sentiment_groups = ['statement']
for group in sentiment_groups:
    for i in range(0, 10):
        sentiment[f'{group}_sentiment_{i}'] = sentiment[f'{group}_sentiment'].shift(i)
        sentiment[f'{group}_sentiment_forward_{i}'] = sentiment[f'{group}_sentiment'].shift(-i)

In [113]:

sentiment.to_csv(f'/Users/kylenabors/Documents/Database/Models/FinBERT Models/taylor.csv', index=False)
